# Exploratory Data Analysis: King County (Washington) real estate sales May 2014-2015

### Connecting and retrieving data via SQLAlchemy

In [ ]:
#import statements
import os
from dotenv import load_dotenv

from sqlalchemy import create_engine

import pandas as pd
import numpy as np
import missingno as msno
#telling pandas to always show all columns
pd.set_option('display.max_columns', None)


In [ ]:
#read the database string from the .env
load_dotenv()

DB_STRING = os.getenv('DB_STRING')

db = create_engine(DB_STRING)

In [ ]:
#import the data to a pandas dataframe
query_string = "SET SCHEMA 'eda'; SELECT * FROM king_county_house_sales AS sales LEFT JOIN king_county_house_details AS details ON sales.house_id = details.id ORDER BY sales.id"
df_sqlalchemy = pd.read_sql(query_string, db)

In [ ]:
df_sqlalchemy.shape

In [ ]:
df_sqlalchemy.head()

### cleaning the data

#### creating a dataframe to work with

In [ ]:
df_housesales = df_sqlalchemy.copy()

In [ ]:
df_housesales.columns

In [ ]:
df_housesales.head()

In [ ]:
df_housesales.info(verbose=True, show_counts=True, max_cols=False)

In [ ]:
#dropping duplicate house id

column_list = df_housesales.columns.tolist()
column_list[4] = "old_id"
df_housesales.columns = column_list

In [ ]:
df_housesales_cleaned = df_housesales.drop('old_id', axis=1)

In [ ]:
#alternative way with drop_duplicates and transposing the dataframe:
df_housesales.T.drop_duplicates().T

In [ ]:
df_housesales_cleaned.head()

In [ ]:
#turning column date to datetime format
df_housesales_cleaned["date"] = pd.to_datetime(df_housesales_cleaned["date"])

#### filling in empty values

In [ ]:
# number of missing values
df_housesales_cleaned.isna().sum()

In [ ]:
msno.bar(df_housesales_cleaned)

In [ ]:
#missing values in waterfront:
print(df_housesales_cleaned["waterfront"].unique())
#no value => no waterfront so fill with 0 and then turn column into boolean
df_housesales_cleaned["waterfront"].fillna(0)
df_housesales_cleaned["waterfront"] = df_housesales_cleaned["waterfront"].astype('bool')
print(df_housesales_cleaned["waterfront"].unique())


In [ ]:
#missing values in view:
print(df_housesales_cleaned["view"].unique())
#no value -> no views, so fill with 0
df_housesales_cleaned["view"] = df_housesales_cleaned["view"].fillna(0)
print(df_housesales_cleaned["view"].unique())


In [ ]:
#missing values in basement, can be calculated by living area - above ground area

df_housesales_cleaned[df_housesales_cleaned["sqft_basement"].isna()][["sqft_living", "sqft_above"]]

In [ ]:
df_housesales_cleaned["sqft_basement"].fillna(df_housesales_cleaned.sqft_living - df_housesales_cleaned.sqft_above, inplace=True)

In [ ]:
#check if no funky math happened:
df_housesales_cleaned[df_housesales_cleaned["sqft_basement"] < 0]

In [ ]:
df_housesales_cleaned2 = df_housesales_cleaned.copy()

In [ ]:
df_housesales_cleaned2["yr_renovated"].unique()

In [ ]:
df_housesales_cleaned2["yr_renovated"] = df_housesales_cleaned2["yr_renovated"].fillna(0)


In [ ]:
df_housesales_cleaned2["yr_renovated"].unique()

In [ ]:
df_housesales_cleaned2["yr_renovated"] = df_housesales_cleaned2["yr_renovated"].apply(lambda x: x/10).astype(int)

In [ ]:
#changing build year to date format
df_housesales_cleaned2["yr_built"] = pd.to_datetime(df_housesales_cleaned2["yr_built"])

In [ ]:
df_housesales_cleaned2["yr_built"].unique() 

In [ ]:
df_housesales_cleaned2.dtypes

### Saving the cleaned dataframe to a backup (pickle)

In [ ]:
df_housesales_cleaned2.to_pickle("data/dataframe_housesales.pkl")